In [1]:
import os

In [2]:
from bs4 import BeautifulSoup


In [ ]:
%pip install beautifulsoup4

In [ ]:
%pip install playwright


In [ ]:
os.system('playwright install')

In [6]:
from playwright.async_api  import async_playwright, TimeoutError as PlaywrightTimeoutError
import time

In [ ]:
%pip install playwright

In [8]:
!playwright install

In [9]:
SEASONS = list(range(2023,2025))

In [ ]:
SEASONS

In [11]:
DATA_DIR = '/Users/sumedhmarathe/Documents/GitHub/NBA_Predictions/data'
STANDINGS_DIR = os.path.join(DATA_DIR, 'standings')
SCORES_DIR = os.path.join(DATA_DIR, 'scores')


In [12]:
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

In [ ]:
%pip show playwright


In [ ]:
!pip show playwright

In [24]:
async def get_html(url, selector,sleep=5, retries=3):
  html = None
  for i in range(1, retries+1):
    (time.sleep(sleep*i))

    try:
      async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        print(await page.title())
        html = await page.inner_html(selector)
    except PlaywrightTimeoutError:
      print('TimeoutError on {url}')
      continue
    else:
      break

  return html



In [25]:
async def scrape_season(season):
  url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'
  html = await get_html(url, '#content .filter')
  soup = BeautifulSoup(html)
  links = soup.find_all('a')
  hrefs = [l["href"] for l in links]
  standings_pages = [f"https://basketball-reference.com{l}" for l in hrefs]

  for url in standings_pages:
    save_path = os.path.join(STANDINGS_DIR, url.split('/')[-1])
    if os.path.exists(save_path):
      continue
    html = await get_html(url, "#all_schedule")
    with open(save_path,'w+') as f:
      f.write(html)

In [ ]:
for season in SEASONS:
  await scrape_season(season)

In [26]:
standings_pages = os.listdir(STANDINGS_DIR)

In [28]:
async def scrape_game(standings_pages):
  with open(standings_pages,'r') as f:
    html = f.read()
    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    hrefs = [l.get('href') for l in links]
    box_scores = [l for l in hrefs if l and "boxscores" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]
    for url in box_scores:
      save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
      if os.path.exists(save_path):
        continue
      html = await get_html(url, "#content")
      if not html:
        continue
      with open(save_path,'w+') as f:
        f.write(html)


In [29]:
import pandas as pd

In [30]:
standing_pages = [s for s in standings_pages if '.html' in s]

In [ ]:

for f in standings_pages:
  filepath = os.path.join(STANDINGS_DIR, f)
  await scrape_game(filepath) 
